In [2]:
#!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

In [4]:
#!apt-get install poppler-utils

In [5]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [ ]:
os.environ["GROQ_API_KEY"] = ""

In [7]:
# Fetch the PDF from the URL
import requests
url = "https://arxiv.org/pdf/2312.10997.pdf"
response = requests.get(url)

In [10]:
# Save the PDF to a local file
with open("types of rag.pdf", "wb") as f:
    f.write(response.content)

In [11]:
# laoding the document
loader = UnstructuredFileLoader("types of rag.pdf")

In [12]:
documents = loader.load()
documents

[Document(metadata={'source': 'types of rag.pdf'}, page_content='4 2 0 2\n\nr a\n\nM 7 2\n\n] L C . s c [\n\n5 v 7 9 9 0 1 . 2 1 3 2 : v i X r a\n\nRetrieval-Augmented Generation for Large Language Models: A Survey\n\nYunfan Gaoa, Yun Xiongb, Xinyu Gaob, Kangxiang Jiab, Jinliu Panb, Yuxi Bic, Yi Daia, Jiawei Suna, Meng Wangc, and Haofen Wang a,c\n\naShanghai Research Institute for Intelligent Autonomous Systems, Tongji University bShanghai Key Laboratory of Data Science, School of Computer Science, Fudan University cCollege of Design and Innovation, Tongji University\n\nAbstract—Large Language Models (LLMs) showcase impres- sive capabilities but encounter challenges like hallucination, outdated knowledge, and non-transparent, untraceable reasoning processes. Retrieval-Augmented Generation (RAG) has emerged as a promising solution by incorporating knowledge from external databases. This enhances the accuracy and credibility of the generation, particularly for knowledge-intensive tasks, 

In [13]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [14]:
texts = text_splitter.split_documents(documents)

In [15]:
type(texts)

list

In [16]:
len(texts)

69

In [19]:
texts[3]

Document(metadata={'source': 'types of rag.pdf'}, page_content='Corresponding Author.Email:haofen.wang@tongji.edu.cn 1Resources RAG-Survey\n\nare\n\navailable\n\nat\n\nhttps://github.com/Tongji-KGLLM/\n\nOur contributions are as follows: • In this survey, we present a thorough and systematic review of the state-of-the-art RAG methods, delineating its evolution through paradigms including naive RAG,\n\n1\n\nFig. 1. Technology tree of RAG research. The stages of involving RAG mainly include pre-training, fine-tuning, and inference. With the emergence of LLMs, research on RAG initially focused on leveraging the powerful in context learning abilities of LLMs, primarily concentrating on the inference stage. Subsequent research has delved deeper, gradually integrating more with the fine-tuning of LLMs. Researchers have also been exploring ways to enhance language models in the pre-training stage through retrieval-augmented techniques.\n\nadvanced RAG, and modular RAG. This review contex- tua

In [20]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-20-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
persist_directory = "doc_db"

In [22]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [23]:
# retriever
retriever = vectordb.as_retriever()

In [38]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [39]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [40]:
# invoke the qa chain and get a response for user query
query = "explain more about modular rag "
response = qa_chain.invoke({"query": query})

In [31]:
print(response)

{'query': 'explain more about modular rag ', 'result': 'Modular RAG is an advanced paradigm of Retrieval-Augmented Generation (RAG) that builds upon the previous Naive RAG and Advanced RAG paradigms. It introduces a more flexible and modular architecture, allowing for greater customization and optimization of the RAG process.\n\nKey characteristics of Modular RAG:\n\n1. **Modular design**: Modular RAG consists of multiple, interchangeable modules that can be combined to form a customized RAG pipeline. This allows for greater flexibility and adaptability to different tasks and domains.\n2. **Functional modules**: Modular RAG introduces specific functional modules that can be used to perform various tasks, such as:\n\t* **Retrieval modules**: responsible for retrieving relevant information from the knowledge base.\n\t* **Generation modules**: responsible for generating text based on the retrieved information.\n\t* **Post-processing modules**: responsible for refining and polishing the ge

In [32]:
print(response["result"])

Modular RAG is an advanced paradigm of Retrieval-Augmented Generation (RAG) that builds upon the previous Naive RAG and Advanced RAG paradigms. It introduces a more flexible and modular architecture, allowing for greater customization and optimization of the RAG process.

Key characteristics of Modular RAG:

1. **Modular design**: Modular RAG consists of multiple, interchangeable modules that can be combined to form a customized RAG pipeline. This allows for greater flexibility and adaptability to different tasks and domains.
2. **Functional modules**: Modular RAG introduces specific functional modules that can be used to perform various tasks, such as:
	* **Retrieval modules**: responsible for retrieving relevant information from the knowledge base.
	* **Generation modules**: responsible for generating text based on the retrieved information.
	* **Post-processing modules**: responsible for refining and polishing the generated text.
3. **Iterative and adaptive retrieval**: Modular RAG 

In [33]:
print(response["source_documents"][0].metadata["source"])

types of rag.pdf


In [34]:
# invoke the qa chain and get a response for user query
query = "Give me summary from this paper?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

The paper appears to be discussing the concept of Retrieval-Augmented Generation (RAG) in the context of natural language processing and question answering. Here's a summary:

The paper presents an overview of RAG, a methodology that combines information retrieval and language generation to answer questions. The RAG process involves three main steps: indexing, retrieval, and generation.

1. Indexing: Documents are split into chunks, encoded into vectors, and stored in a vector database.
2. Retrieval: The top K chunks most relevant to the question are retrieved based on semantic similarity.
3. Generation: The original question and retrieved chunks are input into a language model to generate the final answer.

The paper also discusses various techniques to enhance the RAG process, including:

* Metadata attachments: Adding metadata information to chunks to filter retrieval and ensure freshness of knowledge.
* Structural index: Establishing a hierarchical structure for documents to expedi